In [6]:
# imports
# general
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
from numpy import dot 
from sklearn.metrics import accuracy_score
# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,random_split
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.utils import make_grid
from torch.autograd import Variable
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.autograd import Function
from torch.nn.parameter import Parameter 
# lime
import lime
import lime.lime_tabular
# advertorch 
from advertorch.attacks import LinfPGDAttack
# net classes
from nets_test import Net,CNet
# cpu / gpu
if torch.cuda.is_available():  
    device = 'cuda:0'
else:  
    device = 'cpu'
print('device: ', device)

# functions
def transform_data(data_loader_obj):
    ims = []
    lls = []
    for imgs,lbls in data_loader_obj:
        for img,lbl in zip(imgs,lbls):
            ims.append(img)
            lls.append(lbl)
    ims = torch.stack(ims)
    lls = torch.stack(lls)
    return ims,lls

def show_tensor_images(image_tensor, num_images=25):
    image_tensor = image_tensor.reshape(image_tensor.shape[0],1,28,28).cpu().detach()
    size = (1,28,28)
    image_unflat = image_tensor.detach().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

device:  cuda:0


In [7]:
# new activation functions =0

class Sine_Activation(nn.Module):
    def __init__(self,f=1.):
        super(Sine_Activation,self).__init__()
        self.f = Parameter(torch.tensor(f))
        self.f.requiresgrad = True
 
    def forward(self,x):
        return torch.sin(self.f*x) +torch.ones(x.shape).to(device)+torch.normal(0,.25,x.shape).to(device) #torch.sin(((2.*np.pi)/self.f)*x)
    
    
class Noisey_ReLU(nn.Module):
    def __init__(self):
        super(Noisey_ReLU,self).__init__()
    
    def forward(self,x):
        return torch.max(torch.zeros(x.shape).to(device),x)-1.33*torch.rand(x.shape).to(device)#+torch.normal(0,1,x.shape).to(device)
        

In [8]:
# obtain mnist data and process
batch_size = 128

dataset = MNIST('.', train=True, download=False,transform=transforms.ToTensor())
dataset_test = MNIST('.', train=False, download=False,transform=transforms.ToTensor())

train1, train2 = random_split(dataset,[30000,30000])#DataLoader(dataset, batch_size=batch_size)
data_loader_train1 = DataLoader(train1, batch_size=batch_size)
data_loader_train2 = DataLoader(train2, batch_size=batch_size)
data_loader_test = DataLoader(dataset_test, batch_size=1000)

In [10]:
net_nr = Net([784,256,64,32],activation_func=Noisey_ReLU(),dev=device,path_info='noiseyRelu')
net_nr.train(data_loader_train1)
net_nr.accuracy(data_loader_test)

epoch/epochs: 10/50
accuracy:  77.08333 %
------------------------------
epoch/epochs: 20/50
accuracy:  95.83333 %
------------------------------
epoch/epochs: 30/50
accuracy:  97.91667 %
------------------------------
epoch/epochs: 40/50
accuracy:  97.91667 %
------------------------------
epoch/epochs: 50/50
accuracy:  97.91667 %
------------------------------
saved to ./models/network1_noiseyRelu[784, 256, 64, 32].pth!
model accuracy:  94.83 %


In [11]:
net_nr.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]         200,960
       BatchNorm1d-2                  [-1, 256]             512
       Noisey_ReLU-3                  [-1, 256]               0
       Noisey_ReLU-4                  [-1, 256]               0
       Noisey_ReLU-5                  [-1, 256]               0
       Noisey_ReLU-6                  [-1, 256]               0
            Linear-7                   [-1, 64]          16,448
       BatchNorm1d-8                   [-1, 64]             128
       Noisey_ReLU-9                   [-1, 64]               0
      Noisey_ReLU-10                   [-1, 64]               0
      Noisey_ReLU-11                   [-1, 64]               0
      Noisey_ReLU-12                   [-1, 64]               0
           Linear-13                   [-1, 32]           2,080
      BatchNorm1d-14                   